광고 비용에 따른 수익률 예측 모델

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [9]:
import os
# tensorflow Log 출력 제어
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'

In [10]:
#난수 생성 방식 고정
np.random.seed(100)
tf.random.set_seed(100)

In [11]:
# data Load, drop, split
df = pd.read_csv("data\\data_advertising.csv")
print('원본 데이터 샘플 :')
print(df.head(),'\n')


원본 데이터 샘플 :
   Unnamed: 0     FB    TV  Newspaper  Sales
0           1  230.1  37.8       69.2   22.1
1           2   44.5  39.3       45.1   10.4
2           3   17.2  45.9       69.3    9.3
3           4  151.5  41.3       58.5   18.5
4           5  180.8  10.8       58.4   12.9 



In [12]:
# 의미없는 변수는 삭제합니다.
df = df.drop(columns=['Unnamed: 0'])

In [13]:
"""
1. Sales 변수는 label 데이터로 Y에 저장하고 나머진 featrue X에 저장합니다.
"""
X = df.drop(columns=['Sales'])
Y = df['Sales']

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.3)

In [14]:
"""
2. 학습용 데이터를 tf.data.Dataset 형태로 변환(텐서플로우 신경망 모델의 학습 데이터)
   from_tensor_slices 함수를 사용하여 변환하고 batch를 수행하게 합니다.
"""
train_ds = tf.data.Dataset.from_tensor_slices((train_X.values, train_Y.values))
train_ds = train_ds.shuffle(len(train_X)).batch(batch_size=5)

In [15]:
# 하나의 batch를 뽑아서 feature와 label로 분리합니다.
[(train_features_batch, label_batch)] = train_ds.take(1)

In [16]:
# batch 데이터를 출력합니다.
print('\nFB, TV, Newspaper batch 데이터:\n',train_features_batch)
print('Sales batch 데이터:',label_batch)


FB, TV, Newspaper batch 데이터:
 tf.Tensor(
[[296.4  36.3 100.9]
 [228.   37.7  32. ]
 [  5.4  29.9   9.4]
 [ 57.5  32.8  23.5]
 [240.1   7.3   8.7]], shape=(5, 3), dtype=float64)
Sales batch 데이터: tf.Tensor([23.8 21.5  5.3 11.8 13.2], shape=(5,), dtype=float64)


In [20]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

np.random.seed(100)
tf.random.set_seed(100)

# 데이터를 DataFrame 형태로 불러 옵니다.
df = pd.read_csv("data\\data_advertising.csv")

# DataFrame 데이터 샘플 5개를 출력합니다.
print('원본 데이터 샘플 :')
print(df.head(),'\n')

# 의미없는 변수는 삭제합니다.
df = df.drop(columns=['Unnamed: 0'])

X = df.drop(columns=['Sales'])
Y = df['Sales']

# 학습용 테스트용 데이터로 분리합니다.
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.3)

# Dataset 형태로 변환합니다.
train_ds = tf.data.Dataset.from_tensor_slices((train_X.values, train_Y))
train_ds = train_ds.shuffle(len(train_X)).batch(batch_size=5)


# keras를 활용하여 신경망 모델을 생성합니다.
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=(3,)),
    tf.keras.layers.Dense(1)
    ])

"""
1. 학습용 데이터를 바탕으로 모델의 학습을 수행합니다.
    
step1. compile 메서드를 사용하여 최적화 모델 설정합니다.
       loss는 mean_squared_error, optimizer는 adam으로 설정합니다.
       
step2. fit 메서드를 사용하여 Dataset으로 변환된 학습용 데이터를 학습합니다.
       epochs는 100으로 설정합니다.
"""
model.compile(loss='mean_squared_error', optimizer='adam')
# verbose 인자는 학습 시, 화면에 출력되는 형태를 설정합니다. (0: 표기 없음, 1: 진행 바, 2: 에포크당 한 줄 출력)
history = model.fit(train_ds, epochs=100, verbose=2)
print('\n')

"""
3. evaluate 메서드를 사용하여 테스트용 데이터의 loss 값을 계산합니다.
"""
loss = model.evaluate(test_X, test_Y, verbose=0)

"""
4. predict 메서드를 사용하여 테스트용 데이터의 예측값을 계산합니다.(lable값)
"""
predictions = model.predict(test_X)

# 결과를 출력합니다.
print("테스트 데이터의 Loss 값: ", loss)
for i in range(5):
    print("%d 번째 테스트 데이터의 실제값: %f" % (i, test_Y.iloc[i]))
    print("%d 번째 테스트 데이터의 예측값: %f" % (i, predictions[i][0]))

원본 데이터 샘플 :
   Unnamed: 0     FB    TV  Newspaper  Sales
0           1  230.1  37.8       69.2   22.1
1           2   44.5  39.3       45.1   10.4
2           3   17.2  45.9       69.3    9.3
3           4  151.5  41.3       58.5   18.5
4           5  180.8  10.8       58.4   12.9 

Epoch 1/100


c:\Users\yumin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 - 1s - 21ms/step - loss: 2723.8687
Epoch 2/100
28/28 - 0s - 1ms/step - loss: 549.8625
Epoch 3/100
28/28 - 0s - 1ms/step - loss: 137.8444
Epoch 4/100
28/28 - 0s - 1ms/step - loss: 108.2911
Epoch 5/100
28/28 - 0s - 1ms/step - loss: 92.3173
Epoch 6/100
28/28 - 0s - 1ms/step - loss: 77.4221
Epoch 7/100
28/28 - 0s - 1ms/step - loss: 64.9600
Epoch 8/100
28/28 - 0s - 2ms/step - loss: 54.0165
Epoch 9/100
28/28 - 0s - 2ms/step - loss: 44.1117
Epoch 10/100
28/28 - 0s - 2ms/step - loss: 35.7192
Epoch 11/100
28/28 - 0s - 1ms/step - loss: 28.8961
Epoch 12/100
28/28 - 0s - 1ms/step - loss: 23.3359
Epoch 13/100
28/28 - 0s - 2ms/step - loss: 18.6564
Epoch 14/100
28/28 - 0s - 2ms/step - loss: 15.1841
Epoch 15/100
28/28 - 0s - 1ms/step - loss: 12.4210
Epoch 16/100
28/28 - 0s - 2ms/step - loss: 10.4721
Epoch 17/100
28/28 - 0s - 1ms/step - loss: 8.8113
Epoch 18/100
28/28 - 0s - 1ms/step - loss: 7.6304
Epoch 19/100
28/28 - 0s - 1ms/step - loss: 6.6841
Epoch 20/100
28/28 - 0s - 2ms/step - loss: 6.0112